# SARIF Python 라이브러리를 사용하여 SARIF 파일로 저장된 SAST (Static Application Security Testing) 처리하기

## 1. SARIF Python 라이브러리 설치

In [2]:
!pip install -q sarif-tools
!pip install -q sarif-om

기본적인 정보를 살펴봅니다.

In [14]:
from sarif import loader

path_to_sarif_file = "spotbugs-sarif.json"

sarif_data = loader.load_sarif_file(path_to_sarif_file)
issue_count_by_severity = sarif_data.get_result_count_by_severity()
error_histogram = sarif_data.get_issue_code_histogram("error")
warning_histogram = sarif_data.get_issue_code_histogram("warning")
note_histogram = sarif_data.get_issue_code_histogram("note")

print(f"Issue count by severity: {issue_count_by_severity}")
print(f"Error histogram: {error_histogram}")
print(f"Warning histogram: {warning_histogram}")
print(f"Note histogram: {note_histogram}")

Issue count by severity: {'error': 4, 'warning': 14, 'note': 0}
Error histogram: [('SQL_INJECTION_JDBC Potential JDBC Injection', 2), ('COMMAND_INJECTION Potential Command Injection', 2)]
Warning histogram: [('CIPHER_INTEGRITY Cipher with no integrity', 3), ('PATH_TRAVERSAL_IN Potential Path Traversal (file read)', 3), ('HARD_CODE_KEY Hard coded key', 2), ('PREDICTABLE_RANDOM Predictable pseudorandom number generator', 2), ('ECB_MODE ECB mode is insecure', 1), ('SQL_NONCONSTANT_STRING_PASSED_TO_EXECUTE Nonconstant string passed to execute or addBatch method on an SQL statement', 1), ('SQL_PREPARED_STATEMENT_GENERATED_FROM_NONCONSTANT_STRING A prepared statement is generated from a nonconstant String', 1), ('XXE_DOCUMENT XML parsing vulnerable to XXE (DocumentBuilder)', 1)]
Note histogram: []


SARIF Object Model을 활용하여 SARIF 파일을 파싱해 봅니다.

In [13]:
import json

import attrs
import sarif_om

def get_sarif_class(schema_ref):
    class_name = schema_ref.split('/')[-1]
    class_name = class_name[0].capitalize() + class_name[1:]
    return getattr(sarif_om, class_name)

def get_field_name(schema_property_name, cls):
    for field in attrs.fields(cls):
        if field.metadata.get('schema_property_name') == schema_property_name:
            return field.name
    return schema_property_name

def get_schema_properties(schema, schema_ref):
    cursor = schema
    for part in schema_ref.split('/'):
        if part == '#':
            cursor = schema
        else:
            cursor = cursor[part]
    return cursor['properties']

def materialize(data, cls, schema, schema_ref):
    fields = {}
    extras = {}
    props = get_schema_properties(schema, schema_ref)

    for key, value in data.items():
        field_name = get_field_name(key, cls)

        if key not in props:
            extras[field_name] = value
            continue

        if '$ref' in props[key]:
            schema_ref = props[key]['$ref']
            field_cls = get_sarif_class(schema_ref)
            fields[field_name] = materialize(value, field_cls, schema, schema_ref)

        elif 'items' in props[key]:
            schema_ref = props[key]['items'].get('$ref')
            if schema_ref:
                field_cls = get_sarif_class(schema_ref)
                fields[field_name] = [materialize(v, field_cls, schema, schema_ref) for v in value]
            else:
                fields[field_name] = value
        else:
            fields[field_name] = value

    obj = cls(**fields)
    obj.__dict__.update(extras)
    return obj

with open('spotbugs-sarif.json', 'r') as file:
    data = json.load(file)

with open('sarif-schema-2.1.0.json', 'r') as file:
    schema = json.load(file)

sarif_log = materialize(data, sarif_om.SarifLog, schema, '#')
print(sarif_log)

SarifLog(runs=[Run(tool=Tool(driver=ToolComponent(name='SpotBugs', associated_component=None, contents=['localizedData', 'nonLocalizedData'], dotted_quad_file_version=None, download_uri=None, full_description=None, full_name=None, global_message_strings=None, guid=None, information_uri='https://spotbugs.github.io/', is_comprehensive=None, language='en', localized_data_semantic_version=None, locations=None, minimum_required_localized_data_semantic_version=None, notifications=None, organization=None, product=None, product_suite=None, properties=None, release_date_utc=None, rules=[ReportingDescriptor(id='CIPHER_INTEGRITY', default_configuration=None, deprecated_guids=None, deprecated_ids=None, deprecated_names=None, full_description=None, guid=None, help=None, help_uri='https://find-sec-bugs.github.io/bugs.htm#CIPHER_INTEGRITY', message_strings={'default': {'text': 'The cipher does not provide data integrity.'}}, name=None, properties=PropertyBag(tags=['SECURITY']), relationships=[Reporti